In [21]:
import pandas as pd
import numpy as np
import datetime as dt

# saved the final dataframe to a new csv file
# changed some of the data types
df = pd.read_csv('C:/Users/rm2019/PythonProjects/Finaldata.csv',index_col=0)
df.launched = pd.to_datetime(df.launched)
df.deadline = pd.to_datetime(df.deadline)
df.rtdate = pd.to_datetime(df.rtdate)

df.dtypes

name                      object
category                  object
main_category             object
currency                  object
deadline          datetime64[ns]
goal                     float64
launched          datetime64[ns]
pledged                  float64
state                     object
backers                    int64
country                   object
date_diff                  int64
funding_status            object
rtdate            datetime64[ns]
name_length              float64
fs_binary                  int64
rate                     float64
goal_usd                 float64
pledged_usd              float64
dtype: object

In [22]:
from sklearn.model_selection import train_test_split

# create training and testing data sets
# originally I normalized the data but it made no difference to R sq'd outcome
outcome = pd.DataFrame(df.fs_binary)
# data_norm = ((data - data.mean()) / data.std()
data = df[['backers','date_diff','goal_usd','name_length']]
data_set = data.join(outcome, how='inner')

X_train, y_test = train_test_split(data_set,test_size=0.3)

In [23]:
# Linear regression model takes arrays
# converting dataframe to array
ax1 = np.array(X_train['backers'])
ax2 = np.array(X_train['date_diff'])
ax3 = np.array(X_train['goal_usd'])
ax4 = np.array(X_train['name_length'])
lx1 = np.array(X_train['fs_binary'])
ax1 = ax1.reshape(len(X_train['backers']),1)
ax2 = ax2.reshape(len(X_train['date_diff']),1)
ax3 = ax3.reshape(len(X_train['goal_usd']),1)
ax4 = ax4.reshape(len(X_train['name_length']),1)
lx1 = lx1.reshape(len(X_train['fs_binary']),1)
# horizontally stacking the arrays into new array of arrays
a = np.hstack([ax1,ax2,ax3,ax4])
lbl = np.hstack([lx1])

In [24]:
from sklearn.linear_model import LinearRegression

# my data had a bunch of NaN's causing really problems with the LR formula
# Im replacing the NaN with the column means
col_mean = np.nanmean(a, axis=0)
inds = np.where(np.isnan(a))
a[inds] = np.take(col_mean, inds[1])

# running the training model call regression model
regression_model = LinearRegression()
regression_model.fit(a,lbl)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [26]:
# taking all the regression coef and assigning them to a variable
b = regression_model.coef_
b1 = b[0][0]
b2 = b[0][1]
b3 = b[0][2]
b4 = b[0][3]

In [27]:
#assigning the intercept to a variable
a = regression_model.intercept_

In [28]:
#turning the testing data into a series of array like I did
# with previous training data
ay1 = np.array(y_test['backers'])
ay2 = np.array(y_test['date_diff'])
ay3 = np.array(y_test['goal_usd'])
ay4 = np.array(y_test['name_length'])
ly1 = np.array(y_test['fs_binary'])
ay1 = ay1.reshape(len(y_test['backers']),1)
ay2 = ay2.reshape(len(y_test['date_diff']),1)
ay3 = ay3.reshape(len(y_test['goal_usd']),1)
ay4 = ay4.reshape(len(y_test['name_length']),1)
ly1 = ly1.reshape(len(y_test['fs_binary']),1)
b = np.hstack([ay1,ay2,ay3,ay4])
lbl_y = np.hstack([ly1])

In [29]:
#getting rid of the NaN values like I did previously with the training data
col_mean_y = np.nanmean(b, axis=0)
inds = np.where(np.isnan(b))
b[inds] = np.take(col_mean_y, inds[1])

In [30]:
# models goodness of fit - 4% -absolutely terrible
regression_model.score(b,lbl_y)

0.03828688976344263

In [37]:
# taking the regression formula and feeding it back into the data set
data_set['projection'] = round(a + (b1 * data_set.backers) + (b2 * data_set.date_diff) + (b3 * data_set.goal_usd) + (b4 * data_set.name_length),0)

In [39]:
# where the actual outcome and the projection are the same put 1 otherwise 0
data_set['result'] = np.where(data_set.fs_binary == data_set.projection,1,0)

In [42]:
# figure out the number of accurate projections
data_set.result.sum() / len(data_set.index)

0.6454229983045208